<a href="https://colab.research.google.com/github/deiveleal/data/blob/main/mestrado/ft105/classificacao/ClassificationNaiveBayesKNNEnsemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FT105A - Tópico Interdisciplinar I: Introdução ao Aprendizado de Máquina
### Aluno: Deive Audieres Leal
### RA: 083423


### Tarefa 2: Naïve Bayes, k-NN e Ensembles

#### Link para o repositório: [ClassificationNaiveBayesKNNEnsemble](https://github.com/deiveleal/data/blob/main/mestrado/ft105/classificacao/ClassificationNaiveBayesKNNEnsemble.ipynb)

# Enunciado:

Escolha um pacote de software para classificação de dados que tenha implementações de algoritmos de árvores de decisão, Naïve Bayes e k-NN, e aplique estes três algoritmos ao conjunto de dados Liver Disorder:

* Utilize subamostragem aleatória com 5 repetições para cada algoritmo e apresente o erro de classificação médio de cada um (para os conjuntos de testes);
* Adote uma divisão de 70% dos dados para treinamento e 30% dos dados para teste;
* Faça a amostragem antes de iniciar o treinamento e use os mesmos dados para todos os algoritmos (em cada repetição);

Para cada repetição, monte um ensemble com os classificadores já treinados (via voto majoritário), aplique ao conjunto de testes e apresente o desempenho médio.

ATENÇÃO: Não se esqueça de apresentar no relatório os parâmetros definidos para cada algoritmo (caso existam)!

ATENÇÃO: Verifique na documentação do conjunto de dados como definir o atributo alvo, e explique o procedimento adotado no relatório.

#### Instala o repositório da UCI

In [1]:
# !pip install ucimlrepo

#### Importação das bibliotecas

In [2]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import naive_bayes as nb
from sklearn import neighbors
from ucimlrepo import fetch_ucirepo
from scipy.stats import mode
import numpy as np

#### Importa o conjunto de dados

#### Realiza a busca dos dados


In [3]:
liver_disorders = fetch_ucirepo(id=60)

#### Os dados são um dataframe pandas já separado em features e targets (atributo alvo)

In [4]:
X = liver_disorders.data.features.astype(str)
y = liver_disorders.data.targets.astype(str)

### Mostra o dataframe montado

In [5]:
pd.concat([X,y], axis=1).head(7)

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0
5,98,55,13,17,17,0.0
6,88,62,20,17,9,0.5


##### Cria função com o modelo de árvore de decisão. Foram usados os parâmetros padrões com exceção do criterio que passei a utilizar o 'log_loss' como ganho da informação ao invés do padrão gini. É retornado a acurácia e a previsão do modelo ao final da execução.

In [6]:
def tree_classifier(X_train, X_test, y_train, y_test):
    clf = tree.DecisionTreeClassifier(criterion='log_loss')
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return {"accuracy":accuracy, "prediction":y_pred}

##### Cria função com o modelo knn. Foram usados os parâmetros padrões. É retornado a acurácia e a previsão do modelo ao final da execução.

In [7]:
def knn_classifier(X_train, X_test, y_train, y_test):
    clf = neighbors.KNeighborsClassifier()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return {"accuracy":accuracy, "prediction":y_pred}

##### Cria função com o modelo Naive Bayes. Foram usados os parâmetros padrões. É retornado a acurácia e a previsão do modelo ao final da execução.

In [8]:
def nb_classifier(X_train, X_test, y_train, y_test):
    clf = nb.GaussianNB()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    return {"accuracy":accuracy, "prediction":y_pred}

##### Cria função que recebe os valores de predição e realiza a votação nos valores

In [9]:
def ensemble_voting(predictions):
    predictions = [np.array(prediction).astype(float) for prediction in predictions]
    predictions_ensemble = mode(predictions, axis=0)[0]
    predictions_ensemble = np.squeeze(predictions_ensemble)    
    return predictions_ensemble

#### Treina os classificadores e realiza a previsão com 5 rodadas de treino e classificação

In [10]:
ensemble_source = {}
for execution in range(0, 5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    ensemble_source[f"tree_accuracy_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["accuracy"]
    ensemble_source[f"tree_classification_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["prediction"]
    ensemble_source[f"knn_accuracy_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["accuracy"]
    ensemble_source[f"knn_classification_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["prediction"]
    ensemble_source[f"nb_accuracy_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["accuracy"]
    ensemble_source[f"nb_classification_{execution}"] = tree_classifier(X_train, X_test, y_train, y_test)["prediction"]

#### Cria dataframe com os resultados da classificação

In [11]:
df_ensemble = pd.DataFrame(ensemble_source)

#### Aplica a votação nos valores preditos

In [12]:
predictions_ensemble = ensemble_voting([
    list(df_ensemble["tree_classification_0"]),
    list(df_ensemble["knn_classification_0"]),
    list(df_ensemble["nb_classification_0"]),
    list(df_ensemble["tree_classification_1"]),
    list(df_ensemble["knn_classification_1"]),
    list(df_ensemble["nb_classification_1"]),
    list(df_ensemble["tree_classification_2"]),
    list(df_ensemble["knn_classification_2"]),
    list(df_ensemble["nb_classification_2"]),
    list(df_ensemble["tree_classification_3"]),
    list(df_ensemble["knn_classification_3"]),
    list(df_ensemble["nb_classification_3"]),
    list(df_ensemble["tree_classification_4"]),
    list(df_ensemble["knn_classification_4"]),
    list(df_ensemble["nb_classification_4"])
])

#### Mostra os valores de previsão após votação

In [13]:
pd.Series(predictions_ensemble, name="ensemble")

0      4.0
1      0.5
2      6.0
3      0.5
4      3.0
      ... 
99     0.5
100    2.0
101    8.0
102    0.5
103    2.0
Name: ensemble, Length: 104, dtype: float64